<a href="https://colab.research.google.com/github/ciherraiz/guita/blob/master/SPY_Weekly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install investpy

     |████████████████████████████████| 4.4MB 2.7MB/s 
     |████████████████████████████████| 245kB 32.6MB/s 
     |████████████████████████████████| 5.5MB 28.4MB/s 
  Created wheel for investpy: filename=investpy-0.9.14-cp36-none-any.whl size=4469457 sha256=f296f3a9b0b05bfae593baf07b68bb5e384248056120275d6e0e9dc3cf9ca883
  Stored in directory: /root/.cache/pip/wheels/86/35/ee/fc38f22f1353037b5118cff64b62aeed42f63f2c84135eaaaa
Successfully built investpy
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [19]:
import datetime
from google.colab import files
import investpy

In [27]:
FILENAME='SPY.csv'
NAME = 'SPDR S&P 500'
COUNTRY = 'united states'
FROM_DATE = '01/01/1990'
TO_DATE = datetime.datetime.now().strftime('%d/%m/%Y')

In [28]:
investpy.etfs.get_etf_information(etf=NAME, country=COUNTRY)

,ETF Name,Prev. Close,Todays Range,ROI (TTM),Open,52 wk Range,Dividends (TTM),Volume,Market Cap,Dividend Yield,Average Vol. (3m),Total Assets,Beta,1-Year Change,Shares Outstanding,Asset Class
0,SPDR S&P 500,307.35,299.42 - 306.39,- 0.46%,306.16,218.26 - 339.08,4.03,127961017.0,2.723200e+11,1.86%,116658477.0,1.676500e+11,1.0,2.93%,892432116.0,Equity


In [29]:
data = investpy.etfs.get_etf_historical_data(etf='SPDR S&P 500', country='united states', from_date=FROM_DATE, to_date=TO_DATE)
data.to_csv(FILENAME)
files.download(FILENAME)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>